In [ ]:
import numpy as np
from reacher_environment import ReacherEnvironment

In [ ]:
convert_fn = lambda x: x
env = ReacherEnvironment('./p2_continuous-control/multiagent/Reacher.app', convert_fn)

### Defining Policy-Value Network

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.distributions import beta
import numpy as np
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
class Policy_Network(nn.Module):
    def __init__(self, input_size, action_size):
        super(Policy_Network, self).__init__()
        # outputs beta likelyhood parameters
        self.fc1 = nn.Linear(input_size, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, 2*action_size)
        self.fcalpha = nn.Linear(2*action_size, action_size)
        self.fcbeta = nn.Linear(2*action_size, action_size)
        self.softplus_act = nn.Softplus()
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        a = self.softplus_act(self.fcalpha(x)) + 1.
        b = self.softplus_act(self.fcbeta(x)) + 1.
        beta_dist = beta.Beta(a,b)
        return beta_dist, x

In [ ]:
from reinforce import REINFORCE
import torch.optim as optim
episode = 3000
tmax = 1000
learning_rate = 1e-3
state_dim = 33
action_dim = 4

# Initializing policy network, optimizer and agent
policy_network = Policy_Network(state_dim,action_dim).to(device)
optimizer = optim.Adam(policy_network.parameters(), lr=learning_rate)
reinforce_agent = REINFORCE(policy_network,
                 optimizer, 
                 entropy_reg_schedule = lambda old_value, rewards: 0.995*old_value,
                 entropy_reg_start=0.01,
                 gamma_schedule = lambda old_value, rewards: 1. - 0.998*(1. - old_value),
                 gamma_start = 0.96,
                 target_score=30,
                 target_score_window=100,
                 action_transform = lambda x: -1+2*x,
                 loss_type = "ratio",
                 verbosity = 50)
reinforce_agent.training(env, tmax, tmax, episode)

In [4]:
from ppo import PPO
import torch.optim as optim
learning_rate = 1e-3
episode = 5000
tmax = 1000
state_dim = 33
action_dim = 4

# Initializing policy network, optimizer and agent
policy_network = Policy_Network(state_dim,action_dim).to(device)
optimizer = optim.Adam(policy_network.parameters(), lr=learning_rate)
ppo_agent = PPO(policy_network,
                 optimizer,
                 SGD_steps=4,
                 ppo_policy_epsilon_schedule=lambda old_value, rewards: 0.9996*old_value,
                 ppo_policy_epsilon_start=0.16,
                 entropy_reg_schedule = lambda old_value, rewards: 0.9998*old_value,
                 entropy_reg_start=0.01,
                 gamma_schedule = lambda old_value, rewards: 1.- 0.9998*(1. - old_value),
                 gamma_start = 0.98,
                 target_score=30,
                 target_score_window=100,
                 action_transform = lambda x: -1+2*x,
                 verbosity = 100)
ppo_agent.training(env, tmax, tmax, episode)

 Episode : 100	 Average reward in last 100 episode : 1.86 epsilon_policy : 0.154 gamma : 0.980 entropy_reg : 0.010
 Episode : 200	 Average reward in last 100 episode : 4.58 epsilon_policy : 0.148 gamma : 0.981 entropy_reg : 0.010
 Episode : 300	 Average reward in last 100 episode : 8.26 epsilon_policy : 0.142 gamma : 0.981 entropy_reg : 0.009
 Episode : 400	 Average reward in last 100 episode : 11.73 epsilon_policy : 0.136 gamma : 0.982 entropy_reg : 0.009
 Episode : 500	 Average reward in last 100 episode : 14.32 epsilon_policy : 0.131 gamma : 0.982 entropy_reg : 0.009
 Episode : 600	 Average reward in last 100 episode : 17.39 epsilon_policy : 0.126 gamma : 0.982 entropy_reg : 0.009
 Episode : 700	 Average reward in last 100 episode : 19.98 epsilon_policy : 0.121 gamma : 0.983 entropy_reg : 0.009
 Episode : 800	 Average reward in last 100 episode : 22.90 epsilon_policy : 0.116 gamma : 0.983 entropy_reg : 0.009
 Episode : 900	 Average reward in last 100 episode : 25.55 epsilon_policy :

[0.18149999594315885,
 0.19799999557435513,
 0.2064999953843653,
 0.348499992210418,
 0.3504999921657145,
 0.36049999194219706,
 0.541999987885356,
 0.615999986231327,
 0.526499988231808,
 0.6739999849349261,
 0.7564999830909074,
 0.7814999825321138,
 0.8174999817274511,
 0.685499984677881,
 0.6054999864660203,
 0.9649999784305692,
 0.9664999783970416,
 0.7819999825209379,
 0.8294999814592302,
 0.6979999843984842,
 1.0624999762512743,
 1.0964999754913152,
 0.9499999787658453,
 1.026499977055937,
 0.9384999790228903,
 0.8384999812580645,
 1.0834999757818877,
 0.9509999787434935,
 1.1779999736696483,
 1.0914999756030739,
 1.0169999772682785,
 1.189499973412603,
 0.9949999777600169,
 1.3129999706521631,
 1.0579999763518573,
 1.2494999720714985,
 1.2149999728426337,
 1.2459999721497297,
 1.4999999664723873,
 1.4454999676905573,
 1.5809999646618962,
 1.5059999663382768,
 1.5274999658577144,
 1.4199999682605267,
 1.6544999630190431,
 1.4934999666176736,
 1.5114999662153423,
 1.76499996054917

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.distributions import beta
import numpy as np
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
class Policy_Value_Network(nn.Module):
    def __init__(self, input_size, action_size):
        super(Policy_Value_Network, self).__init__()
        # outputs beta likelyhood parameters
        self.policy_network = Policy_Network(input_size, action_size)
        self.fcval = nn.Linear(2*action_size, 1)
        
    def forward(self, x):
        beta_dist,x = self.policy_network(x)
        values = self.fcval(x)
        return beta_dist, values

In [5]:
from a2c import A2C_PPO_LOSS
from itertools import chain
import torch.optim as optim
learning_rate = 1e-3
episode = 10000
tmax = 1000
state_dim = 33
action_dim = 4

# Initializing policy network, optimizer and agent
policy_value_network = Policy_Value_Network(state_dim,action_dim).to(device)
optimizer = optim.Adam(policy_value_network.parameters(), lr=learning_rate)
a2c_agent = A2C_PPO_LOSS(policy_value_network,
                 optimizer,
                 value_loss_coef=1,
                 n_boot_strap=1,
                 normalize_advantage =True,
                 use_gae_advantage = True,
                 lambda_bootstrap_schedule = lambda old_value, rewards: max([0.9999*old_value,0.3]),
                 lambda_bootstrap_start=0.85,
                 ppo_value_epsilon_schedule = lambda old_value, rewards: max([0.9996*old_value, 0.1]),
                 ppo_value_epsilon_start = 1,
                 SGD_steps=4,
                 ppo_policy_epsilon_schedule=lambda old_value, rewards: max([0.9996*old_value, 0.03]),
                 ppo_policy_epsilon_start=0.15,
                 entropy_reg_schedule = lambda old_value, rewards: 0.9998*old_value,
                 entropy_reg_start=0.01,
                 gamma_schedule = lambda old_value, rewards: 1.- 0.9998*(1. - old_value),
                 gamma_start = 0.98,
                 target_score=30,
                 target_score_window=100,
                 action_transform = lambda x: -1+2*x,
                 verbosity = 100)
a2c_agent.training(env, tmax, tmax, episode)

 Episode : 100	 Average reward in last 100 episode : 0.64 epsilon_policy : 0.144 epsilon_value : 0.961 lambda_gae : 0.842 gamma : 0.980 entropy_reg : 0.010
 Episode : 200	 Average reward in last 100 episode : 1.44 epsilon_policy : 0.138 epsilon_value : 0.923 lambda_gae : 0.833 gamma : 0.981 entropy_reg : 0.010
 Episode : 300	 Average reward in last 100 episode : 3.09 epsilon_policy : 0.133 epsilon_value : 0.887 lambda_gae : 0.825 gamma : 0.981 entropy_reg : 0.009
 Episode : 400	 Average reward in last 100 episode : 6.28 epsilon_policy : 0.128 epsilon_value : 0.852 lambda_gae : 0.817 gamma : 0.982 entropy_reg : 0.009
 Episode : 500	 Average reward in last 100 episode : 8.32 epsilon_policy : 0.123 epsilon_value : 0.819 lambda_gae : 0.809 gamma : 0.982 entropy_reg : 0.009
 Episode : 600	 Average reward in last 100 episode : 9.97 epsilon_policy : 0.118 epsilon_value : 0.787 lambda_gae : 0.800 gamma : 0.982 entropy_reg : 0.009
 Episode : 700	 Average reward in last 100 episode : 13.91 epsil

[0.0859999980777502,
 0.19899999555200337,
 0.2109999952837825,
 0.28699999358505013,
 0.2024999954737723,
 0.25449999431148174,
 0.18299999590963126,
 0.135499996971339,
 0.17799999602138997,
 0.3709999917075038,
 0.1839999958872795,
 0.21749999513849616,
 0.3039999932050705,
 0.21949999509379267,
 0.1804999959655106,
 0.3074999931268394,
 0.2889999935403466,
 0.38399999141693114,
 0.3384999924339354,
 0.40649999091401695,
 0.33299999255686996,
 0.4679999895393848,
 0.5389999879524112,
 0.5604999874718487,
 0.6009999865666031,
 0.5644999873824418,
 0.7284999837167561,
 0.603999986499548,
 0.6744999849237502,
 0.5744999871589244,
 0.6179999861866236,
 0.7729999827221036,
 0.6284999859519302,
 0.5914999867789448,
 0.6654999851249158,
 0.7949999822303653,
 0.8349999813362956,
 0.7569999830797315,
 0.6129999862983823,
 0.677999984845519,
 0.6754999849013984,
 0.9079999797046184,
 0.5314999881200493,
 0.6534999853931367,
 0.6974999844096601,
 0.7669999828562141,
 0.8904999800957739,
 0.442

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.distributions import beta
import numpy as np
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
class Policy_Value_Network(nn.Module):
    def __init__(self, input_size, action_size):
        super(Policy_Value_Network, self).__init__()
        # outputs beta likelyhood parameters
        self.fc1 = nn.Linear(input_size, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fcalpha = nn.Linear(64, action_size)
        self.fcbeta = nn.Linear(64, action_size)
        self.fcval = nn.Linear(64, 1)
        self.softplus_act = nn.Softplus()
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        a = self.softplus_act(self.fcalpha(x)) + 1.
        b = self.softplus_act(self.fcbeta(x)) + 1.
        beta_dist = beta.Beta(a,b)
        value = self.fcval(x)
        return beta_dist, value

### Defining function to collect trajectories

In [ ]:
def collect_trajectories(policy_value, env, tmax):
    states_list = []
    actions_list = []
    rewards_list = []
    log_probs_list = []
    values_list = []
    
    states = env.reset_env()                             # reset the environment    
    scores = np.zeros(env.nagents) 
    t = 0
    while t <= tmax:
        states_tensor = torch.tensor(states, dtype=torch.float32, device=device) 
        states_list.append(states_tensor)                         # storing states
        dists, values = policy_value(states_tensor)        # getting distribution out of the policy network
        
        # getting log prob of sampled actions
        samples = dists.sample()                            # sampling from distribution
        log_probs = torch.sum(dists.log_prob(samples),1)
        log_probs_list.append(log_probs) # storing model probs
        
        # converting samples in [0,1] to actions in [-1,1]
        actions_list.append(samples) 
        actions = -1. + 2. * samples.detach().cpu().numpy()
        
        # generating rewards by interacting with environment
        next_states, rewards, dones = env.take_action(actions)
        rewards_tensor = torch.tensor(rewards, dtype=torch.float32, device=device)[None,:]
        rewards_list.append(rewards_tensor)                # storing rewards
        
        # storing values for each state
        values_list.append(torch.transpose(values,0,1))
        scores += np.array(rewards)                        # update the score (for each agent)
        states = next_states                               # roll over states to next time step
        if np.any(dones):                                  # exit loop if episode finished
            break
        t += 1
        
    next_states_tensor = torch.tensor(next_states, dtype=torch.float32, device=device) 
    states_list.append(next_states_tensor)                        # storing states
    dists, values = policy_value(next_states_tensor)
    dones_tensor = torch.tensor(dones, dtype=torch.float32, device=device)[None,:]
    values = torch.transpose(values,0,1)
    
    values = torch.where(dones_tensor == 0,values, torch.zeros_like(values))
    values_list.append(values)
    rewards_list.append(values)
    
    return torch.cat(states_list,0), torch.cat(actions_list,0), \
            torch.cat(rewards_list,0), torch.cat(values_list,0), torch.cat(log_probs_list,0), scores

In [ ]:
states, actions,rewards, log_prob, values, scores = collect_trajectories(policy_value_network, env, 100)

In [ ]:
states.shape, actions.shape, rewards.shape, log_prob.shape, values.shape, scores.shape

In [ ]:
def total_future_rewards(R, gamma):
    discounts = np.array([gamma**i for i in range(R.shape[0])])
    dis_R = R*discounts[:,None]
    tot_dis_f_R = dis_R[::-1].cumsum(axis=0)[::-1]
    tot_dis_f_R = tot_dis_f_R/discounts[:,None]
    return tot_dis_f_R
    
def kstep_boot_strapped_advantage(Rf, n_bootstrap, V, gamma):
    A = Rf[:-n_bootstrap] + (gamma**n_bootstrap) * (V[n_bootstrap:] - Rf[n_bootstrap:])  \
        - V[:-n_bootstrap]
    return A

In [ ]:
R = rewards
gamma = 0.99
discounts = torch.tensor([[gamma**i] for i in range(R.shape[0])], dtype=torch.float64, device=device)
dis_R = R*discounts
tot_dis_f_R = dis_R + dis_R.sum(dim=0, keepdims=True) - dis_R.cumsum(dim=0)
tot_dis_f_R = tot_dis_f_R/discounts
A = tot_dis_f_R.detach().numpy()

B = total_future_rewards(R.detach().numpy(), gamma)
(A-B).sum()

In [ ]:
states[0].shape, actions[0].shape, rewards[0].shape, log_prob[0].shape

In [ ]:
torch.tensor(dones, dtype=torch.float32, device=device)[None,:].shape

In [ ]:
values[0].shape

In [ ]:
import numpy as np
from collections import deque
# training loop max iterations
episode = 1500
discount_rate = .99
epsilon = 0.1
beta_val = .01
tmax = 1000
SGD_epoch = 4
learning_rate = 1e-4
score_window_len = 100
state_dim = 33
action_dim = 4
max_score = 30
n_bootstrap = 5

# Initializing policy and value network, and optimizer
policy_value_network = Policy_Value(state_dim,action_dim).to(device)

states, actions, rewards, log_probs, values,  scores = \
            collect_trajectories(policy_value_network, env, tmax)
scores.mean()

In [ ]:
print(rewards.shape, values.shape)

### Defining k step booststrap reward

In [ ]:
def total_future_rewards(R, gamma):
    discounts = np.array([gamma**i for i in range(R.shape[0])])
    dis_R = R*discounts[:,None]
    tot_dis_f_R = dis_R[::-1].cumsum(axis=0)[::-1]
    tot_dis_f_R = tot_dis_f_R/discounts[:,None]
    return tot_dis_f_R
    
def kstep_boot_strapped_advantage(Rf, n_bootstrap, V, gamma):
    A = Rf[:-n_bootstrap] + (gamma**n_bootstrap) * (V[n_bootstrap:] - Rf[n_bootstrap:])  \
        - V[:-n_bootstrap]
    return A

### Defining Advantage Actor-Critic update function

In [ ]:
from datetime import datetime
def update_A2C(optimizer, 
               policy_value_network, 
               old_log_probs, 
               states, 
               actions, 
               rewards,
               values,
               gamma = 0.995, 
               epsilon=0.1, 
               beta_val=0.01, 
               n_bootstrap=5):
    
    start = datetime.now()
    # old probs
    old_log_probs = torch.tensor(old_log_probs[:-n_bootstrap+1].flatten(), 
                             dtype=torch.float32, device=device)
        
    # total future reward for value function update
    Rf = total_future_rewards(rewards, gamma)
    RF = torch.tensor(Rf[:-n_bootstrap].flatten(),dtype=torch.float32, device=device)
    
    # advantage calculation for policy update
    A = kstep_boot_strapped_advantage(Rf,n_bootstrap,values,gamma)
    A = torch.tensor(A.flatten(), dtype=torch.float32, device=device)
    
    # actions
    actions = (np.concatenate(actions[:-n_bootstrap+1]) + 1.)/2.
    actions = torch.tensor(actions, dtype=torch.float32, device=device)
    
    # convert states to prob and logprob
    states = torch.cat(states[:-n_bootstrap],0)
    dist,pred_values = policy_value_network(states)
    
    # PPO loss on policy
    new_log_probs = torch.sum(dist.log_prob(actions),1)
    log_ratio = new_log_probs - old_log_probs
    ratio = torch.exp(log_ratio)
    clipped_ratio = torch.clamp(ratio, 1.-epsilon, 1.+epsilon)
    surr_clipped_erew = A*torch.min(ratio, clipped_ratio)
    entropy = torch.sum(dist.entropy(),1)
    policy_loss = -torch.mean(surr_clipped_erew + beta_val*entropy)
    
    # Value function loss
    value_loss = torch.mean(torch.pow(RF - pred_values[:,0],2))
    
    # total loss
    total_loss = policy_loss + value_loss
    
    
    # updating policy network
    optimizer.zero_grad()
    total_loss.backward()
    optimizer.step()
    del total_loss
    
    return policy_value_network

In [ ]:
import numpy as np
from collections import deque
# training loop max iterations
episode = 1500
discount_rate = .99
epsilon = 0.1
beta_val = .01
tmax = 1000
SGD_epoch = 4
learning_rate = 1e-4
score_window_len = 100
state_dim = 33
action_dim = 4
max_score = 30
n_bootstrap = 5

# Initializing policy and value network, and optimizer
policy_value_network = Policy_Value(state_dim,action_dim).to(device)
import torch.optim as optim
optimizer = optim.Adam(policy_value_network.parameters(), lr=learning_rate)

# keep track of progress
mean_rewards = []
mean_rewards_window = deque(maxlen=score_window_len)
for e in range(episode):

    # collect trajectories
    policy_value_network.eval()
    with torch.no_grad():
        states, actions, rewards, log_probs, values,  scores = \
            collect_trajectories(policy_value_network, env, tmax)
        
    # gradient ascent step
    policy_value_network.train()
    with torch.enable_grad()
    for _ in range(SGD_epoch):
        policy_value_network = update_A2C(optimizer, 
               policy_value_network, 
               log_probs, 
               states, 
               actions, 
               rewards,
               values,
               gamma = discount_rate, 
               epsilon=epsilon, 
               beta_val=beta_val, 
               n_bootstrap=n_bootstrap)
        
    # the clipping parameter reduces as time goes on
    epsilon*=.995
    
    # the regulation term also reduces
    # this reduces exploration in later runs
    beta_val*=.995
    
    # get the average reward of the parallel environments
    mean_of_all_agents = np.mean(scores)
    mean_rewards.append(mean_of_all_agents)
    mean_rewards_window.append(mean_of_all_agents)
    avg_of_last_x_episodes = np.mean(mean_rewards_window)
    
    # display some progress every 20 iterations
    print(f"\r Episode : {e+1}\t Average reward in last 100 episode : {avg_of_last_x_episodes:.2f}",end="")
    if (e+1)%50 ==0 :
        print(f"\r Episode : {e+1}\t Average reward in last 100 episode : {avg_of_last_x_episodes:.2f}")
    
    if avg_of_last_x_episodes >= max_score:
        print(f"Environment solved in episodes = {e+1}")

In [ ]:
env.close()